# Importing Gurobi

In [1]:
from gurobipy import*
model = Model("VSOA")

Academic license - for non-commercial use only


# Sets

In [2]:
I = {1}
J = {1,2,3,4}
R = {0,1,2,3}

# Parameter

final weight of supplier j

In [3]:
J,w = multidict({1:0.2273, 2:0.2274, 3:0.2474, 4:0.3049})

defective rate of item i offered by supplier j

In [4]:
J,q = multidict({1:0.08, 2:0.04, 3:0.03, 4:0.01})

on-time delivery rate of item i offered by supplier j

In [5]:
J,t = multidict({1:0.83, 2:0.83, 3:0.95, 4:0.86})

maximum supply capacity of item i offered by supplier j

In [6]:
J,C =  multidict({1:400, 2:700, 3:600, 4:500})

unit price of item i quoted by supplier j

In [7]:
J,p = multidict({1:411, 2:555, 3:629, 4:728})

total demand of item i

In [8]:
I,D = multidict({1:800})

buyer's maximum acceptable defective rate of item i

In [9]:
I,Q = multidict({1:0.02})

buyer's minimum acceptable on-time delivery rate of item i

In [10]:
I,T = multidict({1:0.9})

discount coefficient associated with interval r of supplier j's discount schedule

In [11]:
d = {(1,1):0, (1,2):0.1, (1,3):0.2,
    (2,1):0, (2,2):0.1, (2,3):0.2,
    (3,1):0, (3,2):0.1, (3,3):0.2,
    (4,1):0, (4,2):0.1, (4,3):0.2}

upper limit in interval r of supplier j's discount schedule

In [12]:
b = {(1,0):0, (1,1):10000, (1,2):20000, (1,3):10000000,
    (2,0):0, (2,1):10000, (2,2):20000, (2,3):10000000,
    (3,0):0, (3,1):10000, (3,2):20000, (3,3):10000000,
    (4,0):0, (4,1):10000, (4,2):20000, (4,3):10000000}

# Decision Variable

Binary integer variable, equal to 1 if business volume purchased from supplier j falls on the discount interval r

In [13]:
y = {}
for j in J:
    for r in R:
        y[j,r] = model.addVar(vtype="b", name="y(%s,%s)"%(j,r))

Units of item i to purchase from supplier j

In [14]:
x = {}
for i in I:
    for j in J:
        x[i,j] = model.addVar(lb=0, vtype="c", name="x(%s,%s)"%(i,j))

Business volume purchased from supplier j in discount interval r

In [15]:
v = {}
for j in J:
    for r in R:
        v[j,r] = model.addVar(vtype="c", name="v(%s,%s)"%(j,r))

# Objective Function

Maximizing total weighted quantity of purchasing

In [16]:
model.setObjectiveN(-quicksum(w[j]*x[i,j] for i,j in x),0)

Minimize the total purchase cost

In [17]:
model.setObjectiveN(quicksum((1-d[j,r])*v[j,r] for j in J for r in range(1,4)),1)

Minimize the number of defective items

In [18]:
model.setObjectiveN(quicksum(q[j]*x[i,j] for i,j in x),2)

Maximizing the number of items delivered on time

In [19]:
model.setObjectiveN(-quicksum(t[j]*x[i,j] for i,j in x),3)

In [20]:
model.ModelSense = GRB.MINIMIZE

# Constraints

In [21]:
model.addConstr(quicksum(v[j,r] for r in range(1,4) for j in J) == quicksum(p[j]*x[1,j] for j in J))

<gurobi.Constr *Awaiting Model Update*>

Capacity constraint

In [22]:
for j in J:
    for i in I:
        model.addConstr(x[i,j] <= C[j])

Discount constraint

In [23]:
for j in J:
    model.addConstr(quicksum(y[j,r] for r in range(1,4)) <= 1)

In [24]:
for j in J:
    for r in range(1,4):
        model.addConstr(b[j,r-1]*y[j,r] <= v[j,r])

In [25]:
for j in J:
    for r in range(1,4):
        model.addConstr(v[j,r] <= (b[j,r]-1)*y[j,r])

Demand constraint

In [26]:
for i in I:
    model.addConstr(quicksum(x[i,j] for j in J) == D[i])

Quality constraint

In [27]:
for i in I:
    model.addConstr(quicksum(x[i,j]*q[j] for j in J) <= D[i]*Q[i])

Delivery constraint

In [28]:
for i in I:
    model.addConstr(quicksum((1-t[j])*x[i,j] for j in J) <= (1-T[i])*D[i])

# Results

In [29]:
model.optimize()

Optimize a model with 36 rows, 36 columns and 88 nonzeros
Variable types: 20 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+07]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 36 rows, 36 columns and 88 nonzeros
Variable types: 20 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+07]
  Objective range  [8e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]
Presolve removed 13 

To ensure that the status is optimal

In [39]:
assert model.Status == GRB.Status.OPTIMAL

Creating a query number of multiple objectives and number of solutions

In [40]:
nSolutions = model.SolCount
nObjectives = model.NumObj
print('Problem has', nObjectives, 'objectives')
print('Gurobi found', nSolutions, 'solutions')

Problem has 4 objectives
Gurobi found 1 solutions


Value for each objective function

In [53]:
solutions = []
#for s in range(nSolutions):
#    model.params.SolutionNumber = s
#    print('Solution', s, ':', end='')
for o in range(nObjectives):
    model.params.ObjNumber = o
    print('Objective value ',o+1, ': ', model.ObjNVal, end='')

Changed value of parameter ObjNumber to 0
   Prev: 3  Min: 0  Max: 2000000000  Default: 0
Objective value  1 :  -221.1321212121212Changed value of parameter ObjNumber to 1
   Prev: 0  Min: 0  Max: 2000000000  Default: 0
Objective value  2 :  433764.84848484845Changed value of parameter ObjNumber to 2
   Prev: 1  Min: 0  Max: 2000000000  Default: 0
Objective value  3 :  16.0Changed value of parameter ObjNumber to 3
   Prev: 2  Min: 0  Max: 2000000000  Default: 0
Objective value  4 :  -720.0

In [54]:
solutions.append(model.getAttr('Xn',x))
solutions

[{(1, 1): 0.0,
  (1, 2): 24.2424242424242,
  (1, 3): 363.63636363636374,
  (1, 4): 412.12121212121207}]

In [37]:
EPS = 0.00000001
for (i,j) in x:
    if x[i,j].X > EPS:
        print("Item %1s bought from supplier %1s is: %3s"%(i,j, x[i,j].X))

Item 1 bought from supplier 2 is: 24.2424242424242
Item 1 bought from supplier 3 is: 363.63636363636374
Item 1 bought from supplier 4 is: 412.12121212121207
